# WEEK PREDICTIONS

This notebook simulates a week between two teams by finding the expected count of each category for each player.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")
from base_python_files.api_info import *
from base_python_files.credentials import *
from base_python_files.api_functions import *
from base_python_files.stat_functions import *

In [ ]:
d = retrieve_espn_data(swid,espn_s2,league_id)
data = clean_data(d)

In [ ]:
data["Team"] = data["TeamID"].map(team_dict)


In [ ]:
skater_cats = [
    "goals",
    "assists",
    "plus_minus",
    "penalty_minutes",
    "power play points",
    "FOW",
    "average_time_on_ice",
    "shots_on_goal",
    "hits",
    "blocks",
]

In [ ]:
metric = "30"

skater_data = data[(data["Position"] != "GK")]

played_players = skater_data.copy()

played_players["Team"] = played_players["TeamID"].map(team_dict)
played_players = played_players[
    (played_players["InjuryStatus"] != "INJURY_RESERVE")
    & (played_players["InjuryStatus"] != "OUT")
]

for cat in skater_cats:
    played_players[cat] = played_players[cat + metric]
played_players = played_players[["ID", "Name", "Team", "LeagueTeamID"] + skater_cats]

In [ ]:
# games = pd.read_csv("schedules/this_weeks_games.csv")
# games = pd.read_csv("schedules/next_weeks_games.csv")
games = pd.read_csv("../schedules/penultimate_week.csv")
g_id = 8
opp_id = 3
g_team = played_players[played_players["LeagueTeamID"] == g_id]
opp_team = played_players[played_players["LeagueTeamID"] == opp_id]

In [ ]:
len(g_team), len(opp_team)

In [ ]:
week = 16
team_scores = retrieve_espn_matchup(swid,espn_s2,league_id,week)
# Unused due to delay

In [ ]:
current_opp_scores_list = [0 for _ in skater_cats]
current_g_scores_list = [0 for _ in skater_cats]


current_g_atoi = 0
current_g_players = 1
current_opp_atoi = 0
current_opp_players = 1

current_opp_scores = {
    skater_cats[i]: current_opp_scores_list[i] for i in range(len(skater_cats))
}
current_g_scores = {
    skater_cats[i]: current_g_scores_list[i] for i in range(len(skater_cats))
}


dropped_days_g = {}
dropped_days_opp = {}
swapped_days_g = {}
swapped_days_opp = {}

In [ ]:
# current_g_atoi = 20 * 60 + 3
# current_g_players = 69
# current_opp_atoi = 0 * 60 + 58
# current_opp_players = 68

# current_opp_scores_list = [
#     9,
#     18,
#     0,
#     33,
#     9,
#     78,
#     current_opp_atoi * current_opp_players / 60,
#     67,
#     45,
#     32,
# ]
# current_g_scores_list = [
#     10,
#     14,
#     13,
#     19,
#     3,
#     61,
#     current_g_atoi * current_g_players / 60,
#     75,
#     40,
#     36,
# ]


# current_opp_scores = {
#     skater_cats[i]: current_opp_scores_list[i] for i in range(len(skater_cats))
# }
# current_g_scores = {
#     skater_cats[i]: current_g_scores_list[i] for i in range(len(skater_cats))
# }

In [ ]:
# swapped_days_g = {}
# swapped_days_g["Monday"]=[('Braden Schneider','Cam Fowler'),('Nikita Zadorov','Ben Chiarot')]

In [ ]:
# dropped_days_g['Tuesday']=['Steven Stamkos','Carter Verhaeghe','Alexis Lafreniere','Mika Zibanejad']

In [ ]:
# current_day = "Thursday"
current_day = None
predict_tomorrow = False

In [ ]:
teams = (g_team,opp_team)
scores = (current_g_scores,current_opp_scores)
swapped_days = (swapped_days_g,swapped_days_opp)
dropped_days = (dropped_days_g,dropped_days_opp)
current_players = (current_g_players,current_opp_players)
overall_wins,win_cats,num_players,win_differences = get_week_predictions(teams, played_players,games,skater_cats,scores,swapped_days,dropped_days,current_players,current_day=current_day,predict_tomorrow=predict_tomorrow)

print(overall_wins)
print("Win cats: ", win_cats)
print(num_players)

## Which games are the most important?

In [ ]:
# games_analysis = pd.read_csv("../schedules/this_weeks_games.csv")
games_analysis = pd.read_csv("../schedules/next_weeks_games.csv")
games_analysis = games_analysis[["Day", "Home Team", "Away Team"]]

In [ ]:
def g_team_counts(team):
    return len(g_team[g_team["Team"] == team])


def opp_team_counts(team):
    return len(opp_team[opp_team["Team"] == team])

In [ ]:
games_analysis["Home Team g"] = games_analysis["Home Team"].apply(g_team_counts)
games_analysis["Away Team g"] = games_analysis["Away Team"].apply(g_team_counts)
games_analysis["Home Team opp"] = games_analysis["Home Team"].apply(opp_team_counts)
games_analysis["Away Team opp"] = games_analysis["Away Team"].apply(opp_team_counts)

games_analysis["Home Bias"] = (
    games_analysis["Home Team g"]
    - games_analysis["Home Team opp"]
    - games_analysis["Away Team g"]
    + games_analysis["Away Team opp"]
)
games_analysis["Preferred result"] = games_analysis["Away Team"].where(
    games_analysis["Home Bias"] < 0
)
games_analysis["Preferred result"].loc[games_analysis["Home Bias"] > 0] = (
    games_analysis["Home Team"]
)
games_analysis["Result Importance"] = abs(games_analysis["Home Bias"])
games_analysis["Total players"] = (
    games_analysis["Home Team g"]
    + games_analysis["Away Team g"]
    + games_analysis["Home Team opp"]
    + games_analysis["Away Team opp"]
)

In [ ]:
games_analysis.sort_values(by="Result Importance", ascending=False)